In [1]:
import numpy as np
import pandas as pd

## Setting Up Data

input: first Character 
output : next Character

In [2]:
# Using Kafka txt Metamorphosis ...
# Input+Prev_hidden --> hidden -->output

In [3]:
data=open('kafka.txt','r').read()

In [4]:
data_size=len(data)          # Full Data
chars=list(set(data))        # Set of Characters
char_size=len(chars)       

In [5]:
data_size,char_size

(138417, 81)

In [6]:
#Need to Make of vector with number of unique characters size. if character is an '\n' , then vector is [1,0,0,0,......(81 chars)]

## Dictionaries for Index-Char Conversions

In [7]:
char_to_int={ch:i for i,ch in enumerate(chars)}
int_to_char={i:ch for i,ch in enumerate(chars)}

char_to_int,int_to_char

({'\n': 0,
  ' ': 4,
  '!': 3,
  '"': 5,
  '$': 7,
  '%': 6,
  "'": 8,
  '(': 10,
  ')': 9,
  '*': 11,
  ',': 13,
  '-': 12,
  '.': 15,
  '/': 2,
  '0': 17,
  '1': 16,
  '2': 19,
  '3': 18,
  '4': 21,
  '5': 20,
  '6': 23,
  '7': 22,
  '8': 25,
  '9': 24,
  ':': 27,
  ';': 26,
  '?': 28,
  '@': 30,
  'A': 29,
  'B': 32,
  'C': 31,
  'D': 34,
  'E': 33,
  'F': 36,
  'G': 35,
  'H': 38,
  'I': 37,
  'J': 40,
  'K': 39,
  'L': 42,
  'M': 41,
  'N': 44,
  'O': 43,
  'P': 46,
  'Q': 45,
  'R': 48,
  'S': 47,
  'T': 50,
  'U': 49,
  'V': 52,
  'W': 51,
  'X': 54,
  'Y': 53,
  'a': 55,
  'b': 57,
  'c': 56,
  'd': 59,
  'e': 58,
  'f': 61,
  'g': 60,
  'h': 63,
  'i': 62,
  'j': 65,
  'k': 64,
  'l': 67,
  'm': 66,
  'n': 69,
  'o': 68,
  'p': 71,
  'q': 70,
  'r': 73,
  's': 72,
  't': 75,
  'u': 74,
  'v': 77,
  'w': 76,
  'x': 79,
  'y': 78,
  'z': 80,
  '\xa7': 14,
  '\xc3': 1},
 {0: '\n',
  1: '\xc3',
  2: '/',
  3: '!',
  4: ' ',
  5: '"',
  6: '%',
  7: '$',
  8: "'",
  9: ')',
  10: '

## Vector for a single character 'a'

In [8]:
vector_for_a=np.zeros((char_size,1))  #(81,1)
vector_for_a[char_to_int['a']]=1      # [0,0,0,0,0,0,0,0,0,0.....1,0,0,0,0,0(81 Chars)]

## HyperParameters

In [9]:
learning_rate=1e-1 
hidden_layer_neurons=100
sequence_length=25      # In one time step

## Intialising the Weights And Bias

In [10]:
#Input-Hidden
Wxh=np.random.randn(hidden_layer_neurons,char_size)*0.01   #Transpose of what is expected 100X81 Single I/P: 81X1

In [11]:
Wxh.shape

(100, 81)

In [12]:
#Recurring Weights...Hidden-Hidden
Whh=np.random.randn(hidden_layer_neurons,hidden_layer_neurons)*0.01   #Transpose of what is expected 100X100

In [13]:
Whh.shape

(100, 100)

In [14]:
#Hidden-Output
Why=np.random.randn(char_size,hidden_layer_neurons)*0.01   #Transpose of what is expected 81X100

In [15]:
Why.shape

(81, 100)

In [16]:
#Hidden Layer Bias
bh=np.zeros((hidden_layer_neurons,1))
#Output Bias
by=np.zeros((char_size,1))

In [17]:
def loss_function(inputs,outputs,previous_hidden):
    
    xs,hs,ys,ps={},{},{},{}
    
    hs[-1]=np.copy(previous_hidden)
    
    loss=0
    
    #Forward Pass
    
    for t in range(len(inputs)):
        
        xs[t]=np.zeros((char_size,1))
        
        xs[t][inputs[t]]=1
        
        #print(Wxh.shape,xs[t].shape,Whh.shape,hs[t-1].shape,bh.shape)
        hs[t]=np.tanh(np.dot(Wxh,xs[t])+ np.dot(Whh,hs[t-1]) + bh)
        
        ys[t]=np.dot(Why,hs[t])+ by
        
        ps[t]=np.exp(ys[t])/np.sum(np.exp(ys[t]))
        
        loss+=-np.log(ps[t][outputs[t],0])
        
    #Backward Pass
    
    dWxh,dWhh,dWhy=np.zeros_like(Wxh),np.zeros_like(Whh),np.zeros_like(Why)
    
    dbh,dby=np.zeros_like(bh),np.zeros_like(by)
    
    dhNext=np.zeros_like(hs[0])
    
    for t in reversed(xrange(len(inputs))):
        
        dy = np.copy(ps[t])
        
        #dLi/dfk=pk-1
        dy[outputs[t]] -= 1
        
        dWhy = np.dot(dy,hs[t].T)
        
        dby += dy
        
        dh = np.dot(Why.T,dy) + dhNext
        
        dhRaw = (1-hs[t]*hs[t])*dh
        
        dbh += dhRaw
        
        dWxh += np.dot(dhRaw,xs[t].T)
        
        dWhh += np.dot(dhRaw,hs[t-1].T)
        
        dhNext = np.dot(Whh.T,dhRaw)
        
        for val in [dWxh,dWhy,dWhh,dbh,dby]:
            np.clip(val,-5,5,out=val)
            
        return loss,dWxh,dWhh,dWhy,dbh,dby,hs[len(inputs)-1]

In [18]:
def generateSentence(hidden,index,length):
    x = np.zeros((char_size,1))
    x[index] = 1
    
    generated_text = []
    
    for i in xrange(length):
        
        hidden = np.tanh(np.dot(Wxh,x) + np.dot(Whh,hidden) + bh)
        y = np.dot(Why,hidden)
        p = np.exp(y)/np.sum(np.exp(y))
        
        next_index = np.random.choice(range(char_size),p = p.ravel())
        
        x = np.zeros((char_size,1))
        x[next_index] = 1
        
        generated_text += [next_index]
        
    
    gen_text = ''.join(int_to_char[i] for i in generated_text)
    print("\n"+gen_text+"\n")

In [19]:
hprev = np.zeros((hidden_layer_neurons,1)) # reset RNN memory  
#predict the 200 next characters given 'a'
generateSentence(hprev,char_to_int['a'],200)


Q.PKB1cL@(;
z4)v!zmD4vfpxFA/Sonq�X8Dnxw93Pin (qb3jb;@gYENGCK
6bK(4Kv('ocL%uNbq*zkM%)h@w-wVywbjXa%�L@3EiEf8mex�;Mx49Iom. D8$�yt"U:.WW?
k;K;qYEj7L*1nnF
ag%cgxkA!s%cUkCqHw;B*ql:�ypO)ay�nOt74ioo-,6cbCCg/U



In [20]:
def train(n_steps):
    
    p = 0
    mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    mbh, mby = np.zeros_like(bh), np.zeros_like(by)    
    
    smooth_loss = -np.log(1.0/char_size)*sequence_length                                                                                                                       

    loop = 0
    hprev = np.zeros((hidden_layer_neurons,1)) 
    
    while(loop <= n_steps):
        
        if(loop == 0 or p+sequence_length+1 >= len(data)):
            hprev = np.zeros((hidden_layer_neurons,1)) 
            p = 0
            #Reset Memory at the beginning and end of one set of training data
            
        inputs = [char_to_int[i] for i in data[p:p+sequence_length]]
        targets = [char_to_int[i] for i in data[p+1:p+sequence_length+1]]
        
        #print(hprev.shape)
        loss, dWxh, dWhh, dWhy, dbh, dby, hprev = loss_function(inputs, targets, hprev)
        smooth_loss = smooth_loss * 0.999 + loss * 0.001
            
        if(loop % 1000 == 0):
            print("Iteration: "+str(loop)+" Loss: "+str(smooth_loss))
            generateSentence(hprev,char_to_int['a'],200)
            
        #Adagrad Update
        for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                [dWxh, dWhh, dWhy, dbh, dby],
                                [mWxh, mWhh, mWhy, mbh, mby]):
            mem += dparam * dparam
            param += -learning_rate * dparam / np.sqrt(mem + 1e-8)
            
        p += sequence_length
        loop += 1

In [22]:
train(100000)

Iteration: 0 Loss: 109.813920252

iueg oodk rk bten eomret and iinniGsee coredgded uliyrererdeesgrydtein ie te ifs hedenyet yy eedoe. rus al nlv,or,el'itloooejrend inad Wy Gre toncoule)
er thereriiaw das nidenour fyofxdverishejs er th

Iteration: 1000 Loss: 94.9887224358

, d
iuxg,,Tewt.ohes,yg,yg,ftpholvhegen yihcoueer wemhash
v cac,mh,vcvs s1rghevh,oekthotrvcfvefhrincgh,Stcvh
v
uvhegIvaDa"cincghw"hkgheGlvhonmr- I?G th "hivhxvh,? ,hheh,v 
gn,gn hcvhc?ccmeug,efdaw
?TaT

Iteration: 2000 Loss: 81.2229400425

djH hB 'PServyevxigiamaxnd,manddtGc;Tat?.;vasvo b0T0h Ne fheao,,janiee tomn 
%kB�6gothirO"gj,adBinanj sBropt,dcean
UnQg,giIcnma cTv t Trcuacc,jt 
iGrie hil." kol jlem.pim
0rje.,us wy 0ed vH, -v"i;gs
Y

Iteration: 3000 Loss: 73.9952140641

cu cn wnereiccx, Grceo;crecH Gi-q;ion. wor;i;;y.ct;f,;r 
u
er.,i
sbl-
hhoxnGlgoeloch fsv;g
lingve, chaHnq;clcursydah,h kavxrBFr; G;irre,fve'n;ywfordnmmv;hisimhr,, w"vm'ns Whifs,;y,; 
Reum
A 
t 
hitwiu

Iteration: 4000 Loss: 70.2371276363

lPnhinemar

Iteration: 35000 Loss: 62.4223079071

teod.,,, LHcofFtucCet;weR)t shiy sxUt loD fnthDtuDg0 o0,d

FT'v? bace Padel"r,,, pe
s;c, we?" N,Djb.mabg 
fokd "faw.-ny, -orex")?en,,, ves.yejc.t hodeng, 
H-, wooves, ck Iust 
H,it Gr "senc"HY'ongecsa

Iteration: 36000 Loss: 60.3411133693

d bGcigong, "qU0;
whadcC?")gawlsiigirnd giR RaUgont; hicqpus;inct vat, thotf leg.thecer

Rmilg,hirc" ied c-eKs,,, be1; w"j cwavevqMekc, hanW 1cRedntDf wevt copinftDegs the..,
thlalndum,, av,ihig jf Gc

Iteration: 37000 Loss: 59.5732874839

nvn Hei.?e hive Gfuelagef; aCcas chen"tYGr cGs., bind Gr.r blrig!edtefj)e.,), byd UGr?IP soG ?u3ly Fasga-f, Waxj Gr!chos ,xjFvegixgeur, Ladg,,, her, Gneg;; 
hickung thou'bn 
Cmoge ant,;,, inx"w, Mf ju

Iteration: 38000 Loss: 60.7857432958

recuicJuls iHv, kicmm"t, whengnghuy"of., "vere.Favef;s M he ;ny 
U�7.C
LBum Gx-ojd tDed "0U carcmeE*
Has qucmeats,y, Ne hbo0'df qABJfanged foulGl."; Gfsan cickin'sal tD 
Sor0,  TTiacqnyx.ss; mCibl tov

Iteration: 39000 Loss: 68.4273727184

3v

KeyboardInterrupt: 